<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/02-Vector_Similarity_Search/02.01_RedisVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Search with RedisVL

[Always-on demo](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

This notebook generates vector embeddings using pre-trained `sentence-transformers/all-MiniLM-L6-v2` model from HuggingFace, loads them to Redis and runs Vector Similarity search against Redis database using the Redis Vector Library ([RedisVL](https://www.redisvl.com/)) for Python.



In [ ]:
# Install Redis client and Hugging Face sentence transformers
!pip install -q sentence_transformers redisvl

### Install Redis Stack locally

This installs the Redis Stack database for local usage if you cannot or do not want to create a Redis Cloud database, and it installs the `redis-cli` that will be used for checking database connectivity, etc. 

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


### Setup Redis connection
***Note - Replace Redis connection values if using a Redis Cloud instance!***

In [ ]:
import os
import redis
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
# Replace values above with your own if using Redis Cloud instance
# REDIS_HOST="redis-12110.c82.us-east-1-2.ec2.cloud.redislabs.com"
# REDIS_PORT=12110
# REDIS_PASSWORD="pobhBJP7Psicp2gV0iqa2ZOc1WdXXXXX"

# Shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

# Test Redis connection
!redis-cli $REDIS_CONN PING

### Initialize Embedding Model 
Import a [RedisVL vectorizer](https://www.redisvl.com/user_guide/vectorizers_04.html#vectorizers) to generate text embeddings. For this lab, we are using the Hugging Face Text Vectorizer and selecting the `sentence-transformers/all-MiniLM-L6-v2` [model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2). 

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from redisvl.utils.vectorize import HFTextVectorizer

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

tqdm.pandas()
# Create a vectorizer
# Choose your model from the huggingface website
hf = HFTextVectorizer(model="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Embed a sentence
test = hf.embed("This is a test sentence.")

# Uncomment to see vector embedding output
# test[:10]

### Embedding generation model

Here we are using `sentence-transformers/all-MiniLM-L6-v2` from HuggingFace. https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

***Note - the following block is not needed for this lab, as we already specified our embedding model above with RedisVL.***

In [ ]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# test=model.encode("This is a test sentence.")
# test[:]

### Download data to vectorize and store in Redis
Download 12k+ tweets

In [ ]:
!wget https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv

***Note - make sure to uncomment the `df=df.head(100)` line below if you are using the Free 30MB Redis Cloud database!*** 

In [ ]:
df = pd.read_csv('Labelled_Tweets.csv').drop(columns=['created_at','score'])
# df=df.head(100) #trim dataframe to fit results into 30MB Redis database
df


### Generate Embeddings

Generate vector embeddings within the dataframe. This step can take 2-3 minutes on GPU runtime for all 12k records.

In [ ]:
def text_to_embedding(text):
  return np.array(hf.embed(text), dtype=np.float32).tobytes()

# Generate vector embeddings
df["text_embedding"] = df["full_text"].progress_apply(text_to_embedding)
df.head()

### Define RediSearch Schema
[Define the RediSearch schema](https://redis.io/docs/latest/develop/interact/search-and-query/basic-constructs/schema-definition/) using the [RedisVL syntax](https://redis.io/docs/latest/integrate/redisvl/api/schema/) with the following details: 
- Key Type: `Hash`
- Key Prefix: `tweet:`
- Indexed Fields: 
  - `text`: Full Text
  - `text_embedding`: Vector

In [ ]:
schema = {
    "index": {
        "name": "tweet:idx",
        "prefix": "tweet:",
        "storage_type": "hash", # default setting -- HASH
    },
    "fields": [
        {   "name": "full_text",
            "type": "text",
            "attrs": {
                "no_stem": False,
                "sortable": False
            }
        },
        {
            "name": "text_embedding",
            "type": "vector",
            "attrs": {
                "dims": 384,
                "distance_metric": "cosine",
                "algorithm": "HNSW",
                "datatype": "float32"
            }

        }
    ],
}

### Create index and load data to Redis

In [ ]:
# Clear Redis database (optional)
# redis.flushdb()

from redisvl.index import SearchIndex

index = SearchIndex.from_dict(schema)

index.connect(REDIS_URL)

index.create(overwrite=True)

keys = index.load(df.to_dict('records'))


In [ ]:
# Check how the data is stored in Redis
# !redis-cli $REDIS_CONN keys "*"
# !redis-cli $REDIS_CONN hgetall "tweet::c4b63c9494194d4ea65b7e265bcd2d5f"

## Query the database

[Alway-on Streamlit app](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)


Try queries like:
“Oil”, “Oil Reserves”, “Fossil fuels”

Lexical Full Text search quickly runs out of matches

Vector search continues to discover relevant tweets

In [ ]:
user_query="oil price"
# Queries to try: "oil reserve", "fossil fuels"

In [ ]:
# Using Full Text Index
from redisvl.query import FilterQuery
from redisvl.query.filter import Text

# Exact match filter -- document must contain the exact word doctor
text_filter = Text("full_text") == user_query

filter_query = FilterQuery(
    return_fields=["full_text"],
    filter_expression=text_filter
)

results = index.query(filter_query)

pd.DataFrame(results)

In [ ]:
# Using Vector Index
from redisvl.query import VectorQuery
# from jupyterutils import result_print

query = VectorQuery(
    vector=hf.embed(user_query),
    vector_field_name="text_embedding",
    return_fields=["full_text", "vector_distance"],
    num_results=10
)
results = index.query(query)
pd.DataFrame(results)

#### Cleanup
(Optional) Delete all keys and indexes from the database. 

In [ ]:
###-- FLUSHDB will wipe out the entire database!!! Use with caution --###
# !redis-cli $REDIS_CONN flushdb